# OTIMIZAÇÃO POR COLÔNIA DE FORMIGAS - ENCONTRANDO O MELHOR CAMINHO

O problema será achar o caminho mais curto.

Na natureza, as formigas tendem a seguir caminhos com mais feromônios.

Os feromônios evaporam com o tempo.

Aplicações: roteamento de internet, sistemas de GPS e detecção de bordas de imagens.

Quando uma formiga se move, ela libera feromônios que tendem a atrair mais formigas, fenômeno chamado de estigmergia.

Uma formiga que achou comida num caminho mais curto produzirá uma concentração maior de feromônio no caminho que ela percorreu. Isso irá atrair mais formigas.

Quanto mais formigas seguirem o caminho, maior a concentração de feromônios, fortalecendo ainda mais esse caminho.

![](formigas.jpg)

![](caminho.jpg)

![](ev.jpg)

![](calc.jpg)

### Importante: O cálculo da atualização do valor do feromônio em uma aresta é diferente do cálculo da probabilidade da formiga escolher tal aresta.

1 - De início, é avaliada a probabilidade de escolha de cada aresta.

2 - Ao criar as formigas, é escolhida a aresta inicial e depois as próximas usando as probabilidades calculadas.

3 - É feita a atualização dos feromônios em cada aresta para a população de formigas em análise.

4 - É feita nova avaliação das probabilidades com os feromônios atualizados. 

5 - Eliminação da população de formigas.

6 - Ao criar novas formigas/caminhos, elas serão diferentes da geração anterior, pois a criação de formigas leva em conta o tamanho da aresta (que não muda) e os feromônios presentes (que mudaram).

7 - O ciclo vai se repetindo com o algoritmo escolhendo cada vez mais as arestas mais curtas que são as mais bem avaliadas. Com o tempo, os feromônios nas arestas mais curtas vão aumentar muito, fazendo essas arestas serem cada vez mais escolhidas.

![](f.jpg)

![](prob.jpg)

É preciso tomar cuidado para não considerar como um novo caminho possível (aresta adjacente) uma aresta que volta ao início do caminho ou que irá retroceder o percurso.

![](arestas.jpg)

### O algoritmo irá ler as probabilidades de cada aresta (que dependem do feromônio ali presente) e irá construir o melhor caminho.

In [353]:
import random

ab = ['AB',['BC','BD'],8,1]
ac = ['AC',['BC','BD'],14,1]
ad = ['AD',[],22,1]
bc = ['BC',['CD'],9,1]
cb = ['CB',['BD'],9,1]
bd = ['BD',[],8,1]
cd = ['CD',[],10,1]

arests = [ab,ac,ad,bc,cb,bd,cd]

In [354]:
cd[2]

10

In [355]:
ac[1]

['BC', 'BD']

Função de probabilidade de escolha de aresta adjacente

In [356]:
def probabs(adja):
  dists = []
  fer = []
  for i in adja:
    for j in arests:
      if j[0] == i:
        dists.append(j[2]) # Adicionando a distância
        fer.append(j[3]) # Adicionando o valor do feromônio

  atratividades = []
  cont = 0
  while cont < len(adja):
    atract = fer[cont]*(1/dists[cont]) # Cálculo de feromônio * qualidade do caminho
    atratividades.append(atract)
    cont += 1

  soma = sum(atratividades)
  probs = []
  for i in atratividades:
    prob = (i/soma)
    probs.append(prob)
  return probs


In [357]:
probabs(['BC', 'BD'])

[0.47058823529411764, 0.5294117647058824]

In [358]:
probabs(['AB', 'AC', 'AD']) # Arestas do início

[0.5167785234899329, 0.2953020134228188, 0.18791946308724833]

Escolha da aresta adjacente

![](selec.jpg)

In [359]:
def escolhaAresta(adjs):
  probab = probabs(adjs) # Recebendo as probabilidades de escolha de cada caminho
  limiares = []
  soma = 0
  for i in probab:
    soma += i
    limiares.append(soma) # Criando os intervalos que a roleta irá cair
  r = random.random() # A roleta em si
  cont = 0
  for i in limiares:
    if r > i:
      cont += 1 # Analisando em qual intervalo a roleta parou
  return adjs[cont]

In [360]:
probabs(['AB', 'AC', 'AD'])

[0.5167785234899329, 0.2953020134228188, 0.18791946308724833]

In [361]:
escolhaAresta(['AB', 'AC', 'AD'])

'AB'

Função formiga/caminho/solução

![](arestas.jpg)

In [362]:
def formiga(): # Gera um caminho possível.
  caminho = []
  iniciais = [ab,ac,ad]
  # arests = [ab,ac,ad,bc,cb,bd,cd]

  # Escolhendo a aresta inicial.
  inicial = escolhaAresta(['AB','AC','AD']) # No ponto inicial, AB, AC e AD são as adjacentes.
  caminho.append(inicial)

  if 'D' in caminho[-1]: # Aresta 'AD' vai do início ao fim sem precisar de adjacentes.
    return caminho
  else:
    while True: # Esse while é importante para projetos maiores. 
                # A forma que arests está organizada já garante que a formiga será gerada corretamente com as atualizações de caminho.
                # Se ac for escolhida no início, e depois cd for escolhida, essas duas operações irão acontecer dentro do for.
                # Se a ordem das arestas em arests fosse diferente, o while seria fundamental.
      for i in arests:
        if caminho[-1] == i[0]:
          adj = i[1] # Carregando adjacentes da aresta escolhida.
          if len(adj)==0: # Encerra o for se tiver chegado no fim do caminho com alguma combinação de arestas.
            break
          else:
            adj_random = escolhaAresta(adj) # Escolhendo a próxima aresta a ser percorrida
            caminho.append(adj_random) # Add a aresta escolhida no caminho
      return caminho
      break # Encerrando o while

In [363]:
formiga()

['AC', 'BD']

Comprimento de cada caminho/solução

In [364]:
def comprimento(form):
  soma = 0
  for i in form:
    for j in arests:
      if i == j[0]:
        soma += j[2]
  return soma


In [365]:
comprimento(['AC','CD'])

24

Atualização dos níveis de feromônio: evaporação e adição

![](calc.jpg)

In [366]:
# Evaporação do feromônio
def evaporacao(evap):
  for i in arests:
    i[3] = i[3]*(1-evap)

# Adição do feromônio
def atualiza_ferom(formigas): # Para várias formigas (combinações de arestas) criadas 
  for i in formigas: # Varrendo cada formiga criada para o problema
    ferom = 1/(comprimento(i))
    for j in i: # Varrendo cada aresta de cada formiga
      for k in arests: # Varrendo todas as arestas que existem em arests
        if k[0] == j: # Comparando arestas em arests com cada aresta de cada formiga
          k[3] = k[3] + ferom # Atualizando o feromônio de uma aresta em arests se essa aresta for percorrida por alguma formiga do problema

In [367]:
for i in arests:
    print(i)

['AB', ['BC', 'BD'], 8, 1]
['AC', ['BC', 'BD'], 14, 1]
['AD', [], 22, 1]
['BC', ['CD'], 9, 1]
['CB', ['BD'], 9, 1]
['BD', [], 8, 1]
['CD', [], 10, 1]


In [368]:
evaporacao(0.3)

In [369]:
for i in arests:
    print(i)

['AB', ['BC', 'BD'], 8, 0.7]
['AC', ['BC', 'BD'], 14, 0.7]
['AD', [], 22, 0.7]
['BC', ['CD'], 9, 0.7]
['CB', ['BD'], 9, 0.7]
['BD', [], 8, 0.7]
['CD', [], 10, 0.7]


In [370]:
formigas = [['AB','BD'],['AD']]
atualiza_ferom(formigas)

for i in arests:
  print(i)

['AB', ['BC', 'BD'], 8, 0.7625]
['AC', ['BC', 'BD'], 14, 0.7]
['AD', [], 22, 0.7454545454545454]
['BC', ['CD'], 9, 0.7]
['CB', ['BD'], 9, 0.7]
['BD', [], 8, 0.7625]
['CD', [], 10, 0.7]


### EXECUÇÃO DO ALGORITMO

In [371]:
ab = ['AB',['BC','BD'],8,1]
ac = ['AC',['BC','BD'],14,1]
ad = ['AD',[],22,1]
bc = ['BC',['CD'],9,1]
cb = ['CB',['BD'],9,1]
bd = ['BD',[],8,1]
cd = ['CD',[],10,1]

arests = [ab,ac,ad,bc,cb,bd,cd]

In [372]:
# Quando mais bem avaliada for uma aresta, mais ela será selecionada em escolhaAresta().
# Assim, haverá mais feromônio nas melhores arestas por causa de atualiza_ferom(), o que aumenta a probabilidade
# da aresta, então será selecionada mais em escolhaAresta() nos próximos ciclos.
# Esse procedimento vai continuar no loop até que quando chamar formiga(), as formigas serão a maior parte das arestas com mais feromônios.

for i in range(30):
  evaporacao(0.3) # Atualiza o feromônio. Já houve atualização acima
  formigas = [] # Matando formigas do ciclo 1(2) até o 9(10).
  for j in range(5): # Cria 5 formigas/caminhos 10 vezes. Em cada ciclo, avalia como cada aresta foi avaliada e selecionada. 
                     # Isso vai valorizar as melhores arestas.
    formigas.append(formiga()) # Criando as formigas/soluções/caminhos
  atualiza_ferom(formigas) # Atualiza o feromônio com as formigas que passam em cada aresta

for i in arests: # Exibe como as arestas foram modificadas
  print(i[0],i[3])

for k in formigas: # Exibindo as formigas/caminhos criadas 
  print(k)

AB 1.0404944004153815
AC 0.0006650309395689149
AD 0.00011105160648376636
BC 0.00026428013604783997
CB 2.2539340290692216e-05
BD 1.0408951512189026
CD 0.00026428013604783997
['AB', 'BD']
['AB', 'BD']
['AB', 'BD']
['AB', 'BD']
['AB', 'BD']
